In [2]:
import json

file_dir = "../../condition/CCSCM"

file_id2ent = f"{file_dir}/id2ent.json"
file_ent2id = f"{file_dir}/ent2id.json"
file_id2rel = f"{file_dir}/id2rel.json"
file_rel2id = f"{file_dir}/rel2id.json"

with open(file_id2ent, 'r') as file:
    cond_id2ent = json.load(file)
with open(file_ent2id, 'r') as file:
    cond_ent2id = json.load(file)
with open(file_id2rel, 'r') as file:
    cond_id2rel = json.load(file)
with open(file_rel2id, 'r') as file:
    cond_rel2id = json.load(file)


file_dir = "../../procedure/CCSPROC"

file_id2ent = f"{file_dir}/id2ent.json"
file_ent2id = f"{file_dir}/ent2id.json"
file_id2rel = f"{file_dir}/id2rel.json"
file_rel2id = f"{file_dir}/rel2id.json"

with open(file_id2ent, 'r') as file:
    proc_id2ent = json.load(file)
with open(file_ent2id, 'r') as file:
    proc_ent2id = json.load(file)
with open(file_id2rel, 'r') as file:
    proc_id2rel = json.load(file)
with open(file_rel2id, 'r') as file:
    proc_rel2id = json.load(file)


import csv

condition_mapping_file = "../../../resources/CCSCM.csv"
procedure_mapping_file = "../../../resources/CCSPROC.csv"
drug_file = "../../../resources/ATC.csv"

condition_dict = {}
with open(condition_mapping_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        condition_dict[row['code']] = row['name'].lower()

procedure_dict = {}
with open(procedure_mapping_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        procedure_dict[row['code']] = row['name'].lower()

drug_dict = {}
with open(drug_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['level'] == '5.0':
            drug_dict[row['code']] = row['name'].lower()

In [3]:
cp_id2ent = {}
cp_id2rel = {}

for key in cond_id2ent.keys():
    cp_id2ent[key] = cond_id2ent[key]

for key in proc_id2ent.keys():
    cp_id2ent[str(int(key)+len(cond_id2ent))] = proc_id2ent[key]

cp_ent2id = {value: key for key, value in cp_id2ent.items()}


for key in cond_id2rel.keys():
    cp_id2rel[key] = cond_id2rel[key]

for key in proc_id2rel.keys():
    cp_id2rel[str(int(key)+len(cond_id2rel))] = proc_id2rel[key]

cp_rel2id = {value: key for key, value in cp_id2rel.items()}

In [4]:
out_file_id2ent = f"./id2ent.json"
out_file_ent2id = f"./ent2id.json"
out_file_id2rel = f"./id2rel.json"
out_file_rel2id = f"./rel2id.json"

with open(out_file_id2ent, 'w') as file:
    json.dump(cp_id2ent, file, indent=6)
with open(out_file_ent2id, 'w') as file:
    json.dump(cp_ent2id, file, indent=6)
with open(out_file_id2rel, 'w') as file:
    json.dump(cp_id2rel, file, indent=6)
with open(out_file_rel2id, 'w') as file:
    json.dump(cp_rel2id, file, indent=6)

In [5]:
import numpy as np
import pickle

with open('../../../graphs/condition/CCSCM/entity_embedding.pkl', 'rb') as f:
    cond_ent_emb = pickle.load(f)

with open('../../../graphs/condition/CCSCM/relation_embedding.pkl', 'rb') as f:
    cond_rel_emb = pickle.load(f)

with open('../../../graphs/procedure/CCSPROC/entity_embedding.pkl', 'rb') as f:
    proc_ent_emb = pickle.load(f)

with open('../../../graphs/procedure/CCSPROC/relation_embedding.pkl', 'rb') as f:
    proc_rel_emb = pickle.load(f)

In [6]:
cond_ent_emb.shape, proc_ent_emb.shape, cond_rel_emb.shape, proc_rel_emb.shape

((13778, 1024), (12747, 1024), (2841, 1024), (2464, 1024))

In [7]:
cp_ent_emb = np.concatenate((cond_ent_emb, proc_ent_emb), axis=0)
cp_rel_emb = np.concatenate((cond_rel_emb, proc_rel_emb), axis=0)

In [8]:
cp_ent_emb.shape, cp_rel_emb.shape

((26525, 1024), (5305, 1024))

In [9]:
ent_emb_pkl = "./entity_embedding.pkl"
rel_emb_pkl = "./relation_embedding.pkl"

with open(ent_emb_pkl, "wb") as file:
    pickle.dump(cp_ent_emb, file)

with open(rel_emb_pkl, "wb") as file:
    pickle.dump(cp_rel_emb, file)